In [33]:
import ee
import geemap
import pprint
from matplotlib import pyplot as plt
import pandas as pd
import os

In [34]:
# Initialise (don't need maybe?)
ee.Initialize()
# Configure the pretty printing output & initialize earthengine
pp = pprint.PrettyPrinter(depth=4)

## Parameters (INPUT)

In [35]:
# Parameters

maps_outdir = os.path.join(os.path.expanduser('~'), '../Japan_mapping/results/maps')

In [36]:
Map = geemap.Map(center=[38,138], zoom=6)
Map.add_basemap('HYBRID')
Map

Map(center=[38, 138], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

### Load prefectures shapefile and visualize

In [37]:
# Japan shp is not small, so we ingest it from personal account (otherwise we can upload locally with geemap.shp_to_ee)
japan_shp = 'users/luiscartor/japan_gadm0'
japan = geemap.ee.FeatureCollection(japan_shp)

Map.addLayer(japan, {}, 'Japan')

point = ee.Geometry.Point([38,138])
Map.addLayer(point, {}, 'Point')

# Load prefecture shapefile (from GEE account)
prefs_shp = 'users/luiscartor/japan_gadm1'
prefs = geemap.ee.FeatureCollection(prefs_shp)

Map.addLayer(prefs, {}, 'Prefectures')

### Functions

In [38]:
# Function to add year attribute to image
def addyear(img):
    return img.set('year', ee.Image(img).date().get('year'))

# Function to add Day Of Year attribute to image
def addDOY(img):
    return img.set('DOY', ee.Image(img).date().getRelative('day', 'year'))

# Function to full date
def addDATE(img):
    return ee.Image.constant(img.date().getRelative('day', 'year')).int().updateMask(img.select(0).mask())


# Parameters for masking function (move to preamble)
def landsatmasking(img):

  qa = img.select('pixel_qa')
  # If the cloud bit (5) is set and the cloud confidence (7) is high
  # or the cloud shadow bit is set (3), then it's a bad pixel.
  cloudorsnow = qa.bitwiseAnd(1 << 5). \
    And(qa.bitwiseAnd(1 << 7)). \
    Or(qa.bitwiseAnd(1 << 3)). \
    Or(qa.bitwiseAnd(1 << 4))  
  # Remove edge pixels that don't occur in all bands
  mask2 = img.mask().reduce(ee.Reducer.min())
  return img.updateMask(cloudorsnow.Not()).updateMask(mask2)

# Spectral indices function: adds index as a new band to every image
def addINDICES(img):
    
    ndvi = img.normalizedDifference(['B4', 'B3']).rename('NDVI')
    lswi = img.normalizedDifference(['B4', 'B5']).rename('LSWI')
    ndsi = img.normalizedDifference(['B2', 'B5']).rename('NDSI')    
    # I multiply by scale factor for EVI (DIDNT WORK TO SOLVE HIGH VALUES ISSUE)
    evi = img.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
        'NIR': img.select('B4').multiply(0.0001),
      'RED': img.select('B3').multiply(0.0001),
      'BLUE': img.select('B1').multiply(0.0001)
    }).rename('EVI')
    
    return img.addBands(ndvi).addBands(lswi).addBands(ndsi).addBands(evi)

# Function to add a band of rice classification to each image in the col
# Rule for rice is that LSWI − NDVI > 0 or LSWI − EVI > 0
def addRICE(img):    
    rice = img.expression(
    "(lswi - ndvi > 0) || (lswi - evi > 0) ? 1\
    :2",{'lswi': img.select('LSWI'),'ndvi': img.select('NDVI'),'evi': img.select('EVI')}).rename('RICE')
    
    return img.addBands(rice)

# Apply Masks Function
def landcovermasking(img):
  return img.updateMask(mask1.Not()).updateMask(mask2.Not()).updateMask(mask3.Not()).updateMask(mask4.Not())

# Function to create binary layer for rice class
def classornot(img):
    return img.eq(1)

### Load data tables

In [49]:
# List of prefectures with flooding data
transplantdates_table = '../Japan_mapping/data/transplantdates_average.csv'
transplantdates = pd.read_csv(transplantdates_table, sep=",")

transplantdates.head(10)

,Prefecture,8589,9094,9599,0004,0509,1014,1519
0,Hokkaido,144,144,144,144,144,148,144
1,Aomori,139,138,138,137,140,142,141
2,Iwate,134,136,135,135,136,138,137
3,Miyagi,127,128,126,126,130,133,131
4,Akita,139,139,138,138,139,143,142
5,Yamagata,136,137,136,137,137,140,138
6,Fukushima,135,136,134,134,134,138,137
7,Ibaraki,127,127,126,125,125,127,126
8,Tochigi,136,132,129,127,127,127,127
9,Gunma,171,169,168,164,164,166,164


In [40]:
# List of prefectures with DOY per temperature data
# temperature_table = '../Japan_mapping/data/prefecturetempdata.csv'
# tempdata = pd.read_csv(temperature_table, sep=",")

temperature_table = '../Japan_mapping/data/prefectureDOYmintemp.csv'
tempdata = pd.read_csv(temperature_table, sep=",")

tempdata.head(10)

,Prefecture,Meteostation,1,2,3,4,5,6,7,8,...,356,357,358,359,360,361,362,363,364,365
0,Hokkaido,Sapporo,-5.8,-5.9,-6.0,-6.1,-6.3,-6.4,-6.5,-6.6,...,-5.0,-5.0,-5.1,-5.1,-5.2,-5.3,-5.4,-5.5,-5.6,-5.7
1,Aomori,Aomori,-2.8,-2.9,-3.0,-3.1,-3.2,-3.3,-3.3,-3.4,...,-2.1,-2.2,-2.2,-2.3,-2.3,-2.4,-2.5,-2.5,-2.6,-2.7
2,Iwate,Morioka,-4.2,-4.4,-4.5,-4.6,-4.7,-4.9,-5.0,-5.1,...,-3.2,-3.3,-3.4,-3.4,-3.5,-3.6,-3.7,-3.8,-4.0,-4.1
3,Miyagi,Sendai,-0.7,-0.8,-0.9,-1.0,-1.1,-1.2,-1.2,-1.3,...,0.2,0.2,0.1,0.0,-0.1,-0.2,-0.3,-0.4,-0.5,-0.6
4,Akita,Akita,-1.4,-1.5,-1.6,-1.7,-1.8,-1.9,-2.0,-2.1,...,-0.7,-0.7,-0.8,-0.8,-0.9,-1.0,-1.0,-1.1,-1.2,-1.3
5,Yamagata,Yamagata,-2.3,-2.4,-2.5,-2.6,-2.7,-2.8,-2.9,-3.0,...,-1.4,-1.5,-1.5,-1.6,-1.7,-1.8,-1.9,-2.0,-2.1,-2.2
6,Fukushima,Fukushima,-0.9,-1.0,-1.1,-1.2,-1.2,-1.3,-1.4,-1.4,...,0.0,-0.1,-0.2,-0.3,-0.3,-0.4,-0.5,-0.6,-0.7,-0.8
7,Ibaraki,Mito,-1.6,-1.7,-1.8,-1.8,-1.8,-1.9,-1.9,-2.0,...,-0.7,-0.9,-1.0,-1.1,-1.1,-1.2,-1.3,-1.4,-1.5,-1.6
8,Tochigi,Utsunomiya,-2.1,-2.2,-2.2,-2.3,-2.3,-2.4,-2.4,-2.5,...,-1.3,-1.4,-1.5,-1.6,-1.7,-1.7,-1.8,-1.9,-2.0,-2.1
9,Gunma,Maebashi,-0.1,-0.1,-0.2,-0.3,-0.3,-0.4,-0.4,-0.5,...,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1,0.0


### Create table with DOY with first/last 0/5/10 degrees

In [41]:
# Vectors to store TGS values (first day of certain min temperature)
TGS0S = []; TGS0E = []; TGS5S = []; TGS5E = []; TGS10S = []; TGS10E = [];


# We loop through each prefecture
for i in range(len(tempdata)):
    #print(i)
#i = 0

    # Extracts first DOY with mintemp > 0,5,10
    T0S = next(x for x, val in enumerate(tempdata.iloc[i,2:]) if val > 0)
    T5S = next(x for x, val in enumerate(tempdata.iloc[i,2:]) if val > 5)
    T10S = next(x for x, val in enumerate(tempdata.iloc[i,2:]) if val > 10)

    # Extracts first DOY with mintemp < 0,5,10 after summer (we consider temp values since August (DOY=214))
    # If value does not exist (desired temp falls into next year), we set to last day of year (so to not to use following year image)
    T0E = next((x for x, val in enumerate(tempdata.iloc[i,215:]) if val < 0), 365-214) + 214
    T5E = next((x for x, val in enumerate(tempdata.iloc[i,215:]) if val < 5), 365-214) + 214
    T10E = next((x for x, val in enumerate(tempdata.iloc[i,215:]) if val < 10), 365-214) + 214

    TGS0S.append(T0S); TGS5S.append(T5S); TGS10S.append(T10S); TGS0E.append(T0E); TGS5E.append(T5E); TGS10E.append(T10E); 

tgstable = pd.DataFrame({'Prefecture': tempdata['Prefecture'], 'TGS0S': TGS0S, 'TGS0E': TGS0E, 'TGS5S': TGS5S
                        , 'TGS5E': TGS5E, 'TGS10S': TGS10S, 'TGS10E': TGS10E})
tgstable.head(10)

,Prefecture,TGS0S,TGS0E,TGS5S,TGS5E,TGS10S,TGS10E
0,Hokkaido,90,327,115,302,148,278
1,Aomori,85,339,113,310,144,283
2,Iwate,90,329,116,299,146,278
3,Miyagi,68,360,99,321,128,295
4,Akita,75,349,104,316,133,289
5,Yamagata,81,344,108,311,135,287
6,Fukushima,68,357,99,319,125,293
7,Ibaraki,62,351,96,322,123,297
8,Tochigi,65,348,95,320,120,296
9,Gunma,53,365,91,327,116,301


### Define dataframes and lists used in loop 

In [42]:
# Data frame for storing image availability
availabilitytable = pd.DataFrame([])


# Define periods names: dont' edit (used in dictionary loop later)
periodnames = {'8590','9094','9599','0004','0509','1014','1520'}

# Main routine

## Extract collections

In [45]:
# Landsat 5
L5col = ee.ImageCollection('LANDSAT/LT05/C01/T1_SR') \
    .select('B[1-7]','pixel_qa') \
    .map(addyear) \
    .map(addDOY)
       
# Landsat 7
L7col = ee.ImageCollection('LANDSAT/LE07/C01/T1_SR') \
    .select('B[1-7]','pixel_qa') \
    .map(addyear) \
    .map(addDOY)

# Merge collections
L57col = L5col.merge(L7col)

## 5 Year period collections

In [63]:
# Subset to 5-year periods
L57_8589 = L57col.filterDate('1985-01-01', '1989-12-31')
L57_9094 = L57col.filterDate('1990-01-01', '1994-12-31')
L57_9599 = L57col.filterDate('1995-01-01', '1999-12-31')
L57_0004 = L57col.filterDate('2000-01-01', '2004-12-31')
L57_0509 = L57col.filterDate('2005-01-01', '2009-12-31')
L57_1014 = L57col.filterDate('2010-01-01', '2014-12-31')
L57_1519 = L57col.filterDate('2015-01-01', '2019-12-31')

# Dictionary with periods and collections
periods_dict = {'8589': L57_8589,
              '9094': L57_9094,
              '9599': L57_9599,
              '0004': L57_0004,
              '0509': L57_0509,
              '1014': L57_1014,
              '1519': L57_1519}

transplantdates = transplantdates.iloc[:2]
transplantdates

,Prefecture,8589,9094,9599,0004,0509,1014,1519
0,Hokkaido,144,144,144,144,144,148,144
1,Aomori,139,138,138,137,140,142,141


## Main loop

In [ ]:
# List for storing national rice maps for each period
periodmapslist = list()

# LOOP FOR PERIODS
for periodcol in periods_dict.values():
    #periodcol = L57_8589

    # List for store prefectural maps for current loop period
    prefmapslist = list()
    
    # LOOP FOR PREFECTURES
    for prefecture in transplantdates['Prefecture']:
        
        # Extract prefecture polygon
        prefpolygon = prefs \
            .filter(ee.Filter.eq("NAME_1", prefecture)) 
        # Clip collection to prefecture
        prefcol = periodcol.filterBounds(prefpolygon)
        
        
        
        # PRE-PROCESSING
        prefcol_masked = prefcol.map(landsatmasking)
        # Add Indices
        prefcol_indices = prefcol_masked.map(addINDICES)
        # Subset to spectral indices
        prefcol_ind = prefcol_indices.select('NDVI','LSWI','NDSI','EVI')
        
        
#         # IMAGE AVAILABILITY PER PERIOD
#         # Obtain number of images per year; avoid same date pixels (different tile but same date), we count disctinct dates
#         freq = prefcol_ind.map(addDATE).reduce(ee.Reducer.countDistinct())

#         # Visualize map
#         #Map2.addLayer(freq, {'bands': ['constant_count'],'min':0,'max':50,'palette': ['00FFFF', '0000FF']}, 
#         #             'Im. Frequency period: ' + list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)])

#         # Histogram of number of images per pixel
#         histogram = freq.reduceRegion(**{
#         'reducer': ee.Reducer.histogram(),
#         'geometry': prefpolygon.geometry(),
#         'scale': 30,
#         'maxPixels': 1e9
#         })

#         # Convert histogram (dictionary type) into list
#         hist = histogram.getInfo()
#         hist_list = list(hist.values())

#         # Plot histogram
#         plt.figure()  # This is needed to plot multiple figures in plot
#         plt.bar(hist_list[0]['bucketMeans'], hist_list[0]['histogram'], color='g')
#         plt.ylabel('Number of pixels')
#         plt.xlabel('Number of images')
#         plt.title('# of images per pixel for the period '+ 
#                list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
#                 ' for '+ prefecture)
#         # Save figure
#         plt.savefig('../Japan_mapping/results/figures/imgavailability_period'+
#                  list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
#                  prefecture +'_hist.eps', bbox_inches="tight")

        
        # LUIS OF TOMORROW: ADD PERIODS TO FLOODING DATES!!!! AND ADAPT SCRIPT!!
        # EXTRACT FLOODING PERIODS AND ESTIMATE IMAGE AVAILABILITY
        # Flooding start day is considered the first day of transplanting (TS)
        flood_start = int(transplantdates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][transplantdates[transplantdates['Prefecture']==prefecture].index[0]])
        # MAKE A DIFFERENT TABLE FOR END PERIOD?
        # Flooding last day is considered the end day of transplanting + 30 days !!!!CHECK THE END OF TRANSPLANTING!!!
        flood_end = int(transplantdates[list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]][transplantdates[transplantdates['Prefecture']==prefecture].index[0]] + 30)
      
        # List for each period (year period and irrigation period)
        # Subset to flooding period
        prefcol_flood = prefcol_ind.filter(ee.Filter.gte('DOY', flood_start)).filter(ee.Filter.lte('DOY', flood_end))

        # Subset to non-flooding period
        prefcol_noflood1 = prefcol_ind.filter(ee.Filter.lte('DOY', flood_start))
        prefcol_noflood2 = prefcol_ind.filter(ee.Filter.gte('DOY', flood_end))
        prefcol_noflood = prefcol_noflood1.merge(prefcol_noflood2)        
        
        # Add number of images per season and period into table
        d = {'Prefecture': [prefecture], 
             'Lustrum': [list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]], 
             'AvImFlooding': [prefcol_flood.size().getInfo()], 
             'AvImNonFlooding': [prefcol_noflood.size().getInfo()]} 
        availabilitytable_pref = pd.DataFrame(data=d)

        # Add prefecture to table
        availabilitytable = availabilitytable.append(availabilitytable_pref)

        #print(availabilitytable_pref)
        #print(availabilitytable)

#         # Histogram of availability during flooding period
#         freq = prefcol_flood.reduce(ee.Reducer.countDistinct())
#         # Histogram of number of images per pixel
#         histogram = freq.reduceRegion(**{
#           'reducer': ee.Reducer.histogram(),
#           'geometry': prefpolygon.geometry(),
#           'scale': 30,
#           'maxPixels': 1e9
#         })

#         # Convert histogram (dictionary type) into list
#         hist = histogram.getInfo()
#         hist_list = list(hist.values())

#         # Plot histogram
#         plt.figure()  # This is needed to plot multiple figures in plot
#         plt.bar(hist_list[0]['bucketMeans'], hist_list[0]['histogram'], color='g')
#         plt.ylabel('Number of pixels')
#         plt.xlabel('Number of images')
#         plt.title('# of images per pixel for flooding period during '+ list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
#                  ' for ' + prefecture)
#         # Save figure
#         plt.savefig('../Japan_mapping/results/figures/floodimageavailability_period'+
#                     list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+
#                     '_'+prefecture+'_hist.eps', bbox_inches="tight")

        
        # FILTER COLLECTIONS FOR TEMPERATURE PERIODS
        # Extracting DOY for temperature data
        TGS0S = int(tgstable['TGS0S'][tgstable[tgstable['Prefecture']==prefecture].index[0]])
        TGS0E = int(tgstable['TGS0E'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
        TGS5S = int(tgstable['TGS5S'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
        TGS5E = int(tgstable['TGS5E'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
        TGS10S = int(tgstable['TGS10S'][tgstable[tgstable['Prefecture']==prefecture].index[0]]) 
        TGS10E = int(tgstable['TGS10E'][tgstable[tgstable['Prefecture']==prefecture].index[0]])
        
        
        # Subset to period between first 0 and last 0 min degrees:
        L57_0S0E = prefcol_ind.filter(ee.Filter.gte('DOY', TGS0S)).filter(ee.Filter.lte('DOY', TGS0E))
        # Subset to period between first 5 and last 5 min degrees:
        L57_5S5E = prefcol_ind.filter(ee.Filter.gte('DOY', TGS5S)).filter(ee.Filter.lte('DOY', TGS5E))
        # Subset to period between first 0 and first 10 min degrees:
        L57_0S10S = prefcol_ind.filter(ee.Filter.gte('DOY', TGS0S)).filter(ee.Filter.lte('DOY', TGS10S))
        # Subset to period between first 10 + 40days and last 10 min degrees:
        L57_10S40to5E = prefcol_ind.filter(ee.Filter.gte('DOY', TGS10S+40)).filter(ee.Filter.lte('DOY', TGS10E))
        # Subset to period between last 10 and first 10 (not thermal growing season)
        L57_10E10S1 = prefcol_ind.filter(ee.Filter.lte('DOY', TGS10S))
        L57_10E10S2 = prefcol_ind.filter(ee.Filter.gte('DOY', TGS10E))
        L57_10E10S = L57_10E10S1.merge(L57_10E10S2)

        # Calculate median value for indeces for rice flooding period
        L57_flood_median = prefcol_flood.median()

        
        # APPLY MASKS (based on Dong 2016, RSE)

        # Mask 1) Sparce vegetation: soil, built-up, water body, low vegetated lands. max EVI (T5S-T5E) < 0.6
        # I could use some percentile, instead of max, in order to avoid outliers
        L57_5S5E_median = L57_5S5E.median()
        L57_5S5E_max = L57_5S5E.max()

        mask1 = L57_5S5E_max.select('EVI').lt(0.5)

        # Mask 2) Natural vegetation mask: forests, natural wetlands, grass. Max EVI (T10E-T10S) > 0.4
        L57_10E10S_median = L57_10E10S.median()
        L57_10E10S_max = L57_10E10S.max()

        mask2 = L57_10E10S_median.select('EVI').gt(0.2)

        #SOLVE ISSUE: EVI GIVING VALUES ABOVE 2!

        # Mask 3) Forest mask (JAXA)
        # Starts from 2007
        forestdataset = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/FNF') \
            .filterDate('2017-01-01', '2017-12-31');

        # Forest is the fnf = 1
        forest = forestdataset.first().select('fnf').eq(1);
        # Create forest mask
        mask3 = forest

        # MASK 3 NOT WORKING VERY WELL. ALOS MAP GETS A LOT OF FALSE POSITIVE FOREST (REDUCING RICE FIELDS)

        # Mask 4) Slope: larger than 3deg
        # Call SRTM elevation dataset
        elevdataset = ee.Image('JAXA/ALOS/AW3D30/V2_2');
        elevation = elevdataset.select('AVE_DSM');
        # Obtain 
        slope = ee.Terrain.slope(elevation);

        # Create mask ro slope > 3 deg
        mask4 = slope.gt(3)
        
        # APPLY MASKS    
        prefcol_flood_masked =  prefcol_flood.map(landcovermasking)

        
        # RICE CLASSIFICATION
        # Method 2 for classifying based on % of images
        
        # Add band to each image for rice classification
        prefcol_flood_masked =  prefcol_flood.map(landcovermasking)

        # Add rice classification layer
        prefcol_ricecol = prefcol_flood_masked.map(addRICE).select('RICE')

        # Now we apply the % of images threshold to classify as rice
        # Reduce the number of images with classornot pixels
        riceprop = prefcol_ricecol.map(classornot).mean()

        # Select % of image necessary to assing rice!
        riceclass = riceprop.gte(0.1).clip(prefpolygon)

        # Insert map in list
        prefmapslist.append(riceclass)

        # End of prefectures loop
        
    # Mosaic all prefectural maps for current loop period
    periodricecol = ee.ImageCollection.fromImages(prefmapslist)
    periodmosaic = periodricecol.mosaic()
        
    # Store period mosaic in list
    periodmapslist.append(periodmosaic)
    
#     # Save period national mosaic to disk: TOO LARGE FOR ALL JAPAN!
#     filename = os.path.join(maps_outdir, 'ricemap_'+
#                            list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]+'.tif')
#     geemap.ee_export_image(periodmosaic, filename=filename, region=japan.geometry(), scale=30, file_per_band=False)
    
    # Save period national mosaic to drive: CHANGE REGION TO "japan"
    filename = 'ricemap_'+list(periods_dict.keys())[list(periods_dict.values()).index(periodcol)]
    geemap.ee_export_image_to_drive(periodmosaic, description=filename, folder='GEEexports', region=prefpolygon.geometry(), scale=30)       

Exporting ricemap_8589 ...
Exporting ricemap_9094 ...
Exporting ricemap_9599 ...


In [ ]:
print(availabilitytable)

In [ ]:
Map2 = geemap.Map(center=[38,138], zoom=6)
Map2.add_basemap('HYBRID')
Map2

In [15]:
#print(len(periodmapslist))
prefcol = ee.ImageCollection.fromImages(prefmapslist)
#print(prefcol.size().getInfo())

Map2.addLayer(prefcol, {}, 'lastperiod')

Map2.addLayer(periodmapslist[0], {}, 'period1')
Map2.addLayer(periodmapslist[1], {}, 'period2')
Map2.addLayer(periodmapslist[2], {}, 'period3')
Map2.addLayer(periodmapslist[3], {}, 'period4')
Map2.addLayer(periodmapslist[4], {}, 'period5')
Map2.addLayer(periodmapslist[5], {}, 'period6')
Map2.addLayer(periodmapslist[6], {}, 'period7')

## Split Maps

In [16]:
Map3 = geemap.Map(center=[38,138], zoom=7)
right_layer = geemap.ee_tile_layer(periodmapslist[0], {}, 'Rice 1985-90')
left_layer = geemap.ee_tile_layer(periodmapslist[6], {}, 'Rice 2015-2020')
#Map3.split_map(left_layer, right_layer)
Map3.add_basemap('HYBRID')
Map3.split_map(left_layer='HYBRID', right_layer=right_layer)
Map3

Map(center=[38, 138], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

In [17]:
Map4 = geemap.Map(center=[38,138], zoom=7)
right_layer = geemap.ee_tile_layer(periodmapslist[0], {}, 'Rice 1985-90')
left_layer = geemap.ee_tile_layer(periodmapslist[6], {}, 'Rice 2015-2020')
Map4.split_map(left_layer, right_layer)
Map4

Map(center=[38, 138], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=F…

# Accuracy assessment

In [ ]:
# IMPERVIOUS AND BARREN
# Rule for impervious and barren is LSWI < 0 
def addIMPERV(img):    
    imperv = img.expression(
    "(lswi < 0) ? 1\
    :2",{'lswi': img.select('LSWI')}).rename('IMPERV')
    
    return img.addBands(imperv)

L57_8589_impervcol = L57_5S5E.map(addIMPERV).select('IMPERV')

# Now we apply the % of images threshold to classify as rice
def classornot(img):
    return img.eq(1)

# Reduce the number of images with classornot pixels
impervprop = L57_8589_impervcol.map(classornot).mean()
# Select % of image necessary to assing imperv
impervclass = impervprop.gte(0.5)

# EVERGREEN
# Rule for evergreen is LSWI > 0 
def addEVER(img):    
    ever = img.expression(
    "(lswi > 0) ? 1\
    :2",{'lswi': img.select('LSWI')}).rename('EVER')
    
    return img.addBands(ever)

L57_8589_evercol = L57_8589.map(addEVER).select('EVER')

# Now we apply the % of images threshold to classify ever
def classornot(img):
    return img.eq(1)

# Reduce the number of images with classornot pixels
everprop = L57_8589_evercol.map(classornot).mean()
# Select % of image necessary to assing imperv
everclass = everprop.gte(0.9)